In [1]:
from fpdf import FPDF
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import argparse

from synthpop.recipes.starter2 import Starter
from synthpop.census_helpers import Census
import warnings
warnings.filterwarnings('ignore')

sns.set()

# Synthesis flow

DIAGRAM 0 HERE: synthesize all

## 1. What does it mean to synthesize?

DEFINITION HERE: ESTO ES LO QUE VAMOS A ANALIZAR, PERO QUE SIGNIFICA? REPRESENTA ALGO EN PARTICULAR? COMO SE INTERPRETA ESTA TABLA?

**Here we have a list of synthesized counties for hh and p from the 02 (AK) state**

In [2]:
ls 02/

02_290.pdf           hh_02_195_2013.csv*  p_02_110_2013.csv*
hh_02_013_2013.csv*  hh_02_198_2013.csv*  p_02_122_2013.csv*
hh_02_016_2013.csv*  hh_02_220_2013.csv*  p_02_130_2013.csv*
hh_02_020_2013.csv*  hh_02_230_2013.csv*  p_02_150_2013.csv*
hh_02_050_2013.csv*  hh_02_240_2013.csv*  p_02_164_2013.csv*
hh_02_060_2013.csv*  hh_02_261_2013.csv*  p_02_170_2013.csv*
hh_02_068_2013.csv*  hh_02_270_2013.csv*  p_02_180_2013.csv*
hh_02_070_2013.csv*  hh_02_275_2013.csv*  p_02_185_2013.csv*
hh_02_090_2013.csv*  hh_02_282_2013.csv*  p_02_188_2013.csv*
hh_02_100_2013.csv*  hh_02_290_2013.csv*  p_02_195_2013.csv*
hh_02_105_2013.csv*  p_02_013_2013.csv*   p_02_198_2013.csv*
hh_02_110_2013.csv*  p_02_016_2013.csv*   p_02_220_2013.csv*
hh_02_122_2013.csv*  p_02_020_2013.csv*   p_02_230_2013.csv*
hh_02_130_2013.csv*  p_02_050_2013.csv*   p_02_240_2013.csv*
hh_02_150_2013.csv*  p_02_060_2013.csv*   p_02_261_2013.csv*
hh_02_164_2013.csv*  p_02_068_2013.csv*   p_02_270_2013.csv*
hh_02_170_2013.csv*  p_0

In [3]:
# These files looks like...
hh = pd.read_csv('02/hh_02_290_2013.csv')
p = pd.read_csv('02/p_02_290_2013.csv')

In [4]:
hh.head(3)

,Unnamed: 0,serialno,RT,puma00,puma10,NP,TYPE,BLD,TEN,VEH,...,hh_size,hh_workers,seniors,sf_detached,tenure_mover,cat_id,state,county,tract,block group
0,0,2010000910239,H,400,-9,1,1,2.0,1.0,0.0,...,one,one,no,yes,own not recent,3972,2,290,100,1
1,1,2010000165017,H,400,-9,1,1,2.0,1.0,0.0,...,one,one,no,yes,own not recent,3972,2,290,100,1
2,2,2012001118337,H,-9,400,2,1,2.0,2.0,0.0,...,two,one,no,yes,own not recent,4164,2,290,100,1


In [5]:
p.head(3)

,Unnamed: 0,serialno,SPORDER,puma00,puma10,AGEP,JWTR,RELP,SCH,SCHL,...,ESR,HISP,PERNP,RAC1P,hispanic,person_age,person_sex,race,cat_id,hh_id
0,0,2012000016443,1,-9,400,45,1.0,0,1.0,19.0,...,1.0,1,43200.0,4,no,35 to 60,male,other,138262,112
1,1,2012000016443,2,-9,400,45,NaN,1,1.0,21.0,...,6.0,1,0.0,4,no,35 to 60,female,other,138258,112
2,2,2012000016443,4,-9,400,11,NaN,2,2.0,7.0,...,NaN,1,NaN,4,no,19 and under,female,other,138242,112


## 2. What does these tables represent? 

## Starter 

### Creating `marginals` and `joint distributions` for households and persons 

![Starter](img/Starter.png)

### 2.1. The `acs5` subject table 

In [6]:
# census api key
key = os.environ['CENSUS']

# AK state
state = '02'

# 290 county (Yukon-Koyukuk)
county = '290'

In [8]:
# creates starter object
starter = Starter(key, state, county, acsyear=2013)

In [9]:
# what do we get from starter class?
print([m for m in dir(starter) if not m.startswith('__')])

['acsyear', 'c', 'county', 'get_available_geography_ids', 'get_geography_name', 'get_household_joint_dist_for_geography', 'get_household_marginal_for_geography', 'get_num_geographies', 'get_person_joint_dist_for_geography', 'get_person_marginal_for_geography', 'h_acs', 'h_acs_cat', 'h_pums_cols', 'p_acs', 'p_acs_cat', 'p_pums_cols', 'state', 'tract']


This contains a bunch of methods and variables for a synthesized:

In [10]:
# ...state and county pair
starter.state, starter.county

('02', '290')

##### The `Census` object and his methods:  Tract and Block group acs subject tables

![acs](img/subject_table.png)

As it is shown in the diagram, a first step that occurs when we create the `starter` object is that methods from `Census` constructor are also instantiated. This gives us back some relevant information. First, the `acs` 5 years [estimates tables](https://www.census.gov/data/developers/data-sets/acs-5year.html) by different geographies:

In [11]:
starter.h_acs.head(2)

,NAME,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,...,B08201_002E,B08201_003E,B08201_004E,B08201_005E,B08201_006E,B08202_001E,B08202_002E,B08202_003E,B08202_004E,B08202_005E
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187,52,13,20,11,20,12,1,18,...,119,40,16,3,7,187,89,74,21,2
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336,68,19,42,20,11,27,8,9,...,214,71,30,5,13,336,160,133,37,3


In [12]:
# when we instantiate the Starter class to create starter object we are also creating a census object
c = Census(key, acsyear=2013)

In [13]:
starter.c 

In [14]:
c

In [15]:
# that has his own methods 
print([ m for m in dir(c) if not m.startswith('__')])

['_get_fips_lookup', '_get_pums_relationship', '_query', '_read_csv', '_scale_and_merge', 'acsyear_files', 'base_url', 'block_group_and_tract_query', 'block_group_query', 'c', 'download_household_pums', 'download_population_pums', 'fips_df', 'fips_url', 'pums00_household_base_url', 'pums00_population_base_url', 'pums10_household_base_url', 'pums10_population_base_url', 'pums_cache', 'pums_household_state_base_url', 'pums_population_state_base_url', 'pums_relationship_df', 'pums_relationship_file_url', 'tract_query', 'tract_to_puma', 'try_fips_lookup']


##### Articulating `Census` and  `Starter` methods

As we said, a first thing that takes place when creating `starter` is the generation of `block groups` and `tracts` tables for a group of variables.

In [16]:
# we create and merge both tables with:
c.block_group_and_tract_query

<bound method Census.block_group_and_tract_query of <synthpop.census_helpers.Census object at 0x7f0676aa10b8>>

In the above diagram it is shown that, inside the `Starter` class some methods from census object `c` are called. But something else to be pointed out here is that, this last one `c` is also calling other methods from `census` module. 

We are interested in the `acs5` method - which is inside the core.py module imported from `import census` module -. Let's check it:

In [17]:
# imported methods from census module
print([ m for m in dir(c.c) if not m.startswith('__')])

['ALL', '_acs', 'acs', 'acs1', 'acs1dp', 'acs3', 'acs3dp', 'acs5', 'acs5dp', 'session', 'sf1', 'sf3']


Now, we will define some arbitrary household variables at block group and tract levels to consume `acs5` subject tables.

In [18]:
# block group
presence_of_children_columns = ['B11005_001E', 'B11005_002E', 'B11005_011E']
block_group_size_attr=['B11005_001E'] # this is the total within the block group

# census tract
vehicle_columns = ['B08201_0%02dE' % i for i in range(1, 7)]
tract_size_attr=['B08201_001E'] # this is the total within the tract

In [19]:
tr = c.c.acs5.get(['NAME'] + presence_of_children_columns,
                 geo={'for': "tract:*",
                      'in': 'state: 02 county: 290'}, year=2013)

In [20]:
pd.DataFrame(tr)

,NAME,B11005_001E,B11005_002E,B11005_011E,state,county,tract
0,"Census Tract 1, Yukon-Koyukuk Census Area, Alaska",523.0,140.0,383.0,02,290,000100
1,"Census Tract 2, Yukon-Koyukuk Census Area, Alaska",549.0,155.0,394.0,02,290,000200
2,"Census Tract 3, Yukon-Koyukuk Census Area, Alaska",612.0,251.0,361.0,02,290,000300
3,"Census Tract 4, Yukon-Koyukuk Census Area, Alaska",372.0,151.0,221.0,02,290,000400


In [21]:
bg = c.c.acs5.get(['NAME'] + presence_of_children_columns,
                  geo={'for': "block group:*",
                       'in': 'state: 02 county: 290'}, year=2013)

In [22]:
pd.DataFrame(bg)

,NAME,B11005_001E,B11005_002E,B11005_011E,state,county,tract,block group
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187.0,58.0,129.0,02,290,000100,1
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336.0,82.0,254.0,02,290,000100,2
2,"Block Group 1, Census Tract 2, Yukon-Koyukuk C...",208.0,57.0,151.0,02,290,000200,1
3,"Block Group 2, Census Tract 2, Yukon-Koyukuk C...",341.0,98.0,243.0,02,290,000200,2
4,"Block Group 1, Census Tract 3, Yukon-Koyukuk C...",234.0,108.0,126.0,02,290,000300,1
5,"Block Group 2, Census Tract 3, Yukon-Koyukuk C...",190.0,76.0,114.0,02,290,000300,2
6,"Block Group 3, Census Tract 3, Yukon-Koyukuk C...",188.0,67.0,121.0,02,290,000300,3
7,"Block Group 1, Census Tract 4, Yukon-Koyukuk C...",180.0,82.0,98.0,02,290,000400,1
8,"Block Group 2, Census Tract 4, Yukon-Koyukuk C...",192.0,69.0,123.0,02,290,000400,2


Both tables are merged at the highest disaggregation level (block group). To do it `scale and merge` Census method scales down from tract to block group level as it is shown below:

In [23]:
# tract variables
starter.h_acs[['NAME']+vehicle_columns]

,NAME,B08201_001E,B08201_002E,B08201_003E,B08201_004E,B08201_005E,B08201_006E
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187,119,40,16,3,7
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336,214,71,30,5,13
2,"Block Group 1, Census Tract 2, Yukon-Koyukuk C...",208,51,73,50,21,11
3,"Block Group 2, Census Tract 2, Yukon-Koyukuk C...",341,83,120,83,34,18
4,"Block Group 1, Census Tract 3, Yukon-Koyukuk C...",234,127,58,35,8,4
5,"Block Group 2, Census Tract 3, Yukon-Koyukuk C...",190,103,47,28,7,3
6,"Block Group 3, Census Tract 3, Yukon-Koyukuk C...",188,102,46,28,7,3
7,"Block Group 1, Census Tract 4, Yukon-Koyukuk C...",180,91,46,32,5,2
8,"Block Group 2, Census Tract 4, Yukon-Koyukuk C...",192,98,50,35,5,3


Finally, we build the `acs` table for block group and tracts levels - this process is repeated for persons too, here we only demo households subject tables-.

In [24]:
# the None tract value correspond to all the tracts
h_acs = c.block_group_and_tract_query(block_group_columns=presence_of_children_columns,
                                       tract_columns=vehicle_columns, 
                                       state='02', county='290',
                                       merge_columns=['tract', 'county', 'state'],
                                       block_group_size_attr="B11005_001E",
                                       tract_size_attr="B08201_001E",
                                       tract=None, year=2013)

In [25]:
h_acs.head(2)

,NAME,B11005_001E,B11005_002E,B11005_011E,state,county,tract,block group,B08201_001E,B08201_002E,B08201_003E,B08201_004E,B08201_005E,B08201_006E
0,"Block Group 1, Census Tract 1, Yukon-Koyukuk C...",187,58,129,02,290,000100,1,187,119,40,16,3,7
1,"Block Group 2, Census Tract 1, Yukon-Koyukuk C...",336,82,254,02,290,000100,2,336,214,71,30,5,13


### 2.2. `Categorize` subject tables 

Once we get the subject data for [households](https://github.com/UDST/synthpop/blob/0bc36f8fef036913b416e0d4eb8a3fda79fc70ad/synthpop/recipes/starter2.py#L52-L68) and [persons](https://github.com/UDST/synthpop/blob/0bc36f8fef036913b416e0d4eb8a3fda79fc70ad/synthpop/recipes/starter2.py#L131-L139) variables, the `acs` dataset is categorized by: 

In [26]:
from synthpop import categorizer as cat

In [27]:
cat.categorize

<function synthpop.categorizer.categorize(df, eval_d, index_cols=None)>

This give us back a multindexed table with new names:

In [28]:
h_acs_cat = cat.categorize(h_acs, {("hh_children", "yes"): "B11005_002E",
                                   ("hh_children", "no"): "B11005_011E",
                                   ("hh_cars", "none"): "B08201_002E",
                                   ("hh_cars", "one"): "B08201_003E",
                                   ("hh_cars", "two or more"):
                                        "B08201_004E + B08201_005E + B08201_006E"},
                           index_cols=['state', 'county', 'tract', 'block group'])

In [29]:
h_acs_cat

cat_name                        hh_cars                  hh_children     
cat_value                          none  one two or more          no  yes
state county tract  block group                                          
02    290    000100 1               119   40          26         129   58
                    2               214   71          48         254   82
             000200 1                51   73          82         151   57
                    2                83  120         135         243   98
             000300 1               127   58          47         126  108
                    2               103   47          38         114   76
                    3               102   46          38         121   67
             000400 1                91   46          39          98   82
                    2                98   50          43         123   69

... that can also be obtained by calling it from the `starter` object:

In [30]:
# here the entire table built in starter2
starter.h_acs_cat

cat_name                        hh_age_of_head           hh_cars       \
cat_value                            gt35-lt65 gt65 lt35    none  one   
state county tract  block group                                         
02    290    000100 1                      122   25   40     119   40   
                    2                      210   77   49     214   71   
             000200 1                      123   67   18      51   73   
                    2                      229   76   36      83  120   
             000300 1                      146   43   45     127   58   
                    2                      121   44   25     103   47   
                    3                      112   37   39     102   46   
             000400 1                       92   49   39      91   46   
                    2                      108   46   38      98   50   

cat_name                                    hh_children        hh_income  \
cat_value                       two or more          no  yes gt100-lt150   
state county tract  block group                                            
02    290    000100 1                    26         129   58           6   
                    2                    48         254   82          33   
             000200 1                    82         151   57           8   
                    2                   135         243   98          59   
             000300 1                    47         126  108           9   
                    2                    38         114   76          41   
                    3                    38         121   67           8   
             000400 1                    39          98   82           3   
                    2                    43         123   69          16   

cat_name                               ... hispanic_head     seniors      \
cat_value                       gt150  ...            no yes      no yes   
state county tract  block group        ...                                 
02    290    000100 1               0  ...           187   0     162  25   
                    2              15  ...           331   5     259  77   
             000200 1               2  ...           204   4     138  70   
                    2              25  ...           339   2     254  87   
             000300 1               1  ...           234   0     184  50   
                    2              16  ...           188   2     143  47   
                    3               0  ...           188   0     147  41   
             000400 1               7  ...           180   0     131  49   
                    2               6  ...           188   4     144  48   

cat_name                        sf_detached        tenure_mover             \
cat_value                                no  yes own not recent own recent   
state county tract  block group                                              
02    290    000100 1                     0  187            135          9   
                    2                    20  316            222         10   
             000200 1                    15  193            147          4   
                    2                    44  297            240         11   
             000300 1                     1  233            150          9   
                    2                     6  184            129         13   
                    3                     4  184            135          2   
             000400 1                     0  180            121          6   
                    2                     7  185             97          8   

cat_name                                                     
cat_value                       rent not recent rent recent  
state county tract  block group                              
02    290    000100 1                        31          12  
                    2                        46          58  
             000200 1                        45   

### 2.3. Geography relations for joint distributions

![pums](img/download_pums.png)

As we can see in the diagram, one important method from `Census` constructor is the:

In [31]:
c.tract_to_puma

<bound method Census.tract_to_puma of <synthpop.census_helpers.Census object at 0x7f0676aa10b8>>

... which returns the correspondant puma id for a given tract:

In [32]:
for tract in ['000100','000200','000300','000400']:
    print('puma10 id for tract {} : {}'.format(tract, c.tract_to_puma(state, county, tract)[0]))
    print('puma00 id for tract {} : {}'.format(tract, c.tract_to_puma(state, county, tract)[1]))
    puma10, puma00 = c.tract_to_puma(state, county, tract)

puma10 id for tract 000100 : 00400
puma00 id for tract 000100 : 00400
puma10 id for tract 000200 : 00400
puma00 id for tract 000200 : 00400
puma10 id for tract 000300 : 00400
puma00 id for tract 000300 : 00400
puma10 id for tract 000400 : 00400
puma00 id for tract 000400 : 00400


This information will be mainly used to download `pums` from `aws` or `gcs` (depending on the acs requested year).

##### What we call slices?: the `PUMA` geographies 

![pums_rel](img/pums_tracts.png)

PUMS stands for Public Use Microdata Sample. These are individual records of survey responses with identifying
information removed.These files do not include every record of every person who responded to the ACS. Only a select few that in turn, are representative of the population. 

The ACS samples 3.5 million addresses per year. The 1-year ACS PUMS file contains about 1% of all of the US households. The 5-year ACS PUMS file is the equivalent of five 1-year files, so it includes about 5% of all of the US households.

By contrast, in aggregated tables or summary data, the individual records are categorized and weighted to create an estimate for the larger population. These estimates contains a Margin of Error (which is, to put it into simple words, the percentage of times we do not hit a target population type when we randomly select cases: e.g. 5%, etc. ).

This means that, since `PUMS` microdata provides a sample of the ACS records it is necessary to create an estimate of how many persons/households the raw records represent.  This microdata has no geographies smaller than what we call `PUMAs`.

PUMA is an area where the population of over 100,000 in – population of over 100,000, large enough to meet the disclosure avoidance requirements. It is identified by a five-digit code that is unique within each estate, and they nest within state or state equivalence.

PUMAs are redefined after each Decennial Census. It is important to mention that PUMAs redefined after the 2010 Census were first used in 2012 ACS PUMS files. Multi-year files contain dual PUMAs vintages, for example, the 2010 to 2014 ACS PUMS files. PUMAs are built on Census Tracks and Counties, and can be combined to create rough approximations of towns, counties or cities for analysis.

In [33]:
# PUMS variables
h_pums_cols = ('serialno', 'PUMA00','PUMA10', 'RT', 'NP', 'TYPE',
               'R65', 'HINCP', 'VEH', 'MV', 'TEN', 'BLD', 'R18')

In [34]:
h_pums = c.download_household_pums(state, puma10, puma00, usecols=h_pums_cols)

Reading PUMS00 from https://s3-us-west-1.amazonaws.com/synthpop-data2/


In [35]:
h_pums

,serialno,RT,puma00,puma10,NP,TYPE,BLD,TEN,VEH,HINCP,MV,R18,R65
0,2012000000643,H,-9,400,2,1,2.0,3.0,0.0,33360.0,1.0,0.0,0.0
2,2012000001889,H,-9,400,4,1,2.0,2.0,0.0,89420.0,7.0,0.0,2.0
3,2012000003083,H,-9,400,1,1,2.0,2.0,0.0,20800.0,7.0,0.0,0.0
4,2012000004753,H,-9,400,7,1,2.0,2.0,0.0,26300.0,6.0,1.0,0.0
5,2012000005450,H,-9,400,3,1,3.0,3.0,1.0,65300.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5857,2011001481195,H,400,-9,6,1,2.0,3.0,1.0,165600.0,7.0,1.0,1.0
5858,2011001486432,H,400,-9,5,1,2.0,2.0,0.0,23900.0,5.0,1.0,0.0
5862,2011001495660,H,400,-9,3,1,2.0,2.0,2.0,29500.0,7.0,0.0,2.0
5865,2011001496296,H,400,-9,1,1,2.0,4.0,0.0,900.0,7.0,0.0,0.0


##### Using `PUMS` files

Joint distributions represents a total value for acs queried table. Since this last dataset contains aggregated data for tracts and block groups levels, and given that the PUMS are a representative sample of individual records - each serial number corresponds to a unique answer -, both tables will be used to build target values to be joined during the synthesis. 

In [36]:
cat.joint_distribution

<function synthpop.categorizer.joint_distribution(sample_df, category_df, mapping_functions=None)>

First, we build a dataframe with the target categories that we obtained querying the `acs subject table`. This, by calling the `category_combinations` method from the categorizer that will return all possible combinations: 

In [37]:
cat.category_combinations(h_acs_cat.columns)

cat_id
hh_cars     hh_children        
none        no                0
            yes               1
one         no                2
            yes               3
two or more no                4
            yes               5

With this, the `joint_distribution` method will return a sample and categories dataframes. This by following next steps:

In [38]:
# mapping functions to return values depending on slices dataframes
def cars_cat(r):
    if r.VEH == 0:
        return "none"
    elif r.VEH == 1:
        return "one"
    return "two or more"

def children_cat(r):
    if r.R18 == 1:
        return "yes"
    return "no"

In [39]:
h_pums, jd_households = cat.joint_distribution(h_pums,
                                                cat.category_combinations(h_acs_cat.columns),
                                                {"hh_cars": cars_cat,
                                                 "hh_children": children_cat,
                                                 })

- **1. Categories dataframe**

This cointains the amount of cases (or frequencies) for each category combination within the PUMA geography. It is important to remark that this totals corresponds to all the tracts we passed to the `tract to puma` function. Given that PUMS returns individual answers for a bunch of variables, we can combine them based on acs subject table columns and get the totals of each combination for a group tracts. This will return a total value for a group of tracts (or PUMA). PUMAs are unique whitin a state.

In [40]:
jd_households

cat_id  frequency
hh_cars     hh_children                   
none        no                0        843
            yes               1        919
one         no                2        573
            yes               3        431
two or more no                4        491
            yes               5        368

- **2. Sample dataframe**

This is the microdata puma df we downloaded for tracts 100 to 400, with a new `cat_id` combination column:

In [41]:
h_pums

,serialno,RT,puma00,puma10,NP,TYPE,BLD,TEN,VEH,HINCP,MV,R18,R65,hh_cars,hh_children,cat_id
0,2012000000643,H,-9,400,2,1,2.0,3.0,0.0,33360.0,1.0,0.0,0.0,none,no,0
2,2012000001889,H,-9,400,4,1,2.0,2.0,0.0,89420.0,7.0,0.0,2.0,none,no,0
3,2012000003083,H,-9,400,1,1,2.0,2.0,0.0,20800.0,7.0,0.0,0.0,none,no,0
15,2012000017291,H,-9,400,4,1,2.0,2.0,0.0,54800.0,5.0,0.0,0.0,none,no,0
31,2012000041419,H,-9,400,2,1,2.0,2.0,0.0,33300.0,7.0,0.0,1.0,none,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,2011001370081,H,400,-9,1,1,2.0,2.0,6.0,300300.0,5.0,0.0,0.0,two or more,no,4
5771,2011001380825,H,400,-9,2,1,2.0,4.0,2.0,173600.0,1.0,0.0,0.0,two or more,no,4
5835,2011001457691,H,400,-9,2,1,1.0,3.0,2.0,97400.0,3.0,0.0,0.0,two or more,no,4
5843,2011001464975,H,400,-9,4,1,2.0,1.0,2.0,8650.0,5.0,0.0,0.0,two or more,no,4


**Until this point, we...**:
1. Queried the `acs5` subject table to get persons and households variables at block group and tracts levels
2. Downloaded sample files (PUMS) containing the same variables with different standard names.
3. Matched the tracts of the `county`, `state` pair with a puma id.
4. Built combination of variables at puma level - or all the tracts that are cointained inside a puma id (e.g. households with no children and no cars)

### 2.4. Marginals, the block group level

Marginals are called from the `starter` object inside the `synthesize_all` function and returns...

In [128]:
#...for a group of available geographies inside a county/state pair
list(starter.get_available_geography_ids())[0]

state              02
county            290
tract          000100
block group         1
dtype: object

In [129]:
indexes = starter.get_available_geography_ids()

In [130]:
for geog_id in indexes:
    h_marg = starter.get_household_marginal_for_geography(geog_id)

***...the total value of a given variable for the block group geography levels:***

In [131]:
# This is the marginal table we stored for the last census tract (400)
h_marg

cat_name         cat_value      
hh_age_of_head   gt35-lt65          108
                 gt65                46
                 lt35                38
hh_cars          none                98
                 one                 50
                 two or more         43
hh_children      no                 123
                 yes                 69
hh_income        gt100-lt150         16
                 gt150                6
                 gt30-lt60           32
                 gt60-lt100          81
                 lt30                57
hh_race_of_head  asian                0
                 black                2
                 other              108
                 white               82
hh_size          four or more        36
                 one                 61
                 three               44
                 two                 51
hh_workers       none                69
                 one                 77
                 two or more         45
hispani

For example, in our acs5 categorized table, in the block group 1 of the 000400 tract (we only stored the last geography since we ran a for loop) there are 82 households with childs and 46 with one car.

In [132]:
# can check these totals in our acs subject table...
h_acs_cat

cat_name                        hh_cars                  hh_children     
cat_value                          none  one two or more          no  yes
state county tract  block group                                          
02    290    000100 1               119   40          26         129   58
                    2               214   71          48         254   82
             000200 1                51   73          82         151   57
                    2                83  120         135         243   98
             000300 1               127   58          47         126  108
                    2               103   47          38         114   76
                    3               102   46          38         121   67
             000400 1                91   46          39          98   82
                    2                98   50          43         123   69

In [134]:
# store the block group 1 of the 000400 census tract
block_gp_1 = h_acs_cat.loc[tuple(list(starter.get_available_geography_ids())[7])]

In [135]:
block_gp_1

cat_name     cat_value  
hh_cars      none           91
             one            46
             two or more    39
hh_children  no             98
             yes            82
Name: (02, 290, 000400, 1), dtype: int64

## 3. Sinthesize all: using `Starter` outputs 

DIAGRAM HERE: how Starter is used by the synthesized all function

In [136]:
from synthpop.ipf.ipf import calculate_constraints

In [137]:
h_constraint, _ = calculate_constraints(block_gp_1, jd_households.frequency)

In [138]:
h_constraint

hh_cars      hh_children
none         no             46.529753
             yes            46.538428
one          no             27.835805
             yes            19.209650
two or more  no             23.634442
             yes            16.251921
dtype: float64

In [139]:
_

4